In [ ]:
set.seed(13)
n <- 1000

X1 <- rbinom(n, 1, 0.8)
X2 <- rnorm(n, 0, 1)
X3 <- rnorm(n, 0, 1)
X4 <- rnorm(n, 0, 1)

D <- rbinom(n, 1, 0.5)
error <- rnorm(n, 0, 1)

Y <- 2*D + 0.5*X1 - 0.3*X2 + 0.2*X3 + error

df <- data.frame(X1, X2, X3, X4, D, Y)
head(df)

In [ ]:
cat("Con t-test\n")
vars <- c("X1", "X2", "X3", "X4")
resultados_ttest <- data.frame()

for (var in vars) {
  t_test <- t.test(df[df$D == 1, var], df[df$D == 0, var])
  resultados_ttest <- rbind(resultados_ttest,
    data.frame(
      Variable = var,
      Tratamiento = mean(df[df$D == 1, var]),
      Control = mean(df[df$D == 0, var]),
      P_value = t_test$p.value
    )
  )
}
print(resultados_ttest)

In [ ]:
cat("Con regresión\n")
resultados_regresion <- data.frame()

for (var in vars) {
  modelo <- lm(df[[var]] ~ D, data = df)
  resumen <- summary(modelo)
  resultados_regresion <- rbind(resultados_regresion,
    data.frame(
      Variable = var,
      Coeficiente = coef(resumen)["D", "Estimate"],
      Error_std = coef(resumen)["D", "Std. Error"],
      P_value = coef(resumen)["D", "Pr(>|t|)"]
    )
  )
}
print(resultados_regresion)

In [ ]:
modelo_1 <- lm(Y ~ D, data = df)
modelo_2 <- lm(Y ~ D + X1 + X2 + X3 + X4, data = df)

summary(modelo_1)
summary(modelo_2)

cat("Resultado sin controles:", coef(modelo_1)["D"], "\n")
cat("Resultado con controles:", coef(modelo_2)["D"], "\n")
cat("Error estándar sin controles:", summary(modelo_1)$coefficients["D","Std. Error"], "\n")
cat("Error estándar con controles:", summary(modelo_2)$coefficients["D","Std. Error"], "\n")

In [ ]:
library(glmnet)

X_cov <- as.matrix(df[, c("X1", "X2", "X3", "X4")])
y <- df$Y

set.seed(13)
cv_lasso <- cv.glmnet(X_cov, y, alpha = 1, nfolds = 10)

coeficientes <- coef(cv_lasso, s = "lambda.min")
print(coeficientes)

In [ ]:
seleccionadas <- rownames(coeficientes)[coeficientes[,1] != 0]
seleccionadas <- seleccionadas[seleccionadas != "(Intercept)"]
cat("Covariables seleccionadas en λmin:", seleccionadas, "\n")